<a href="https://colab.research.google.com/github/yeajinleeee/Project_ORI-Team_movie_review/blob/main/LYJ_movie_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [6]:
import pandas as pd

# Attempt to read the file, handling potential errors
try:
    data = pd.read_csv('/content/drive/MyDrive/DNA/2024 여름방학 프로젝트/IMDB 영화 리뷰/IMDB Dataset.csv')
except pd.errors.ParserError as e:
    print(f"Error reading CSV: {e}")

    # Inspect the problematic row and surrounding rows
    with open('/content/drive/MyDrive/DNA/2024 여름 프로젝트/IMDB 영화 리뷰/IMDB Dataset.csv', 'r') as file:
        for i, line in enumerate(file):
            if i >= 11037 and i <= 11041:  # Check a few lines before and after
                print(f"Line {i}: {line.strip()}")

In [8]:
data.drop_duplicates(inplace=True)

In [9]:
duplicate_rows = data[data.duplicated()]
print("중복된 값:\n", duplicate_rows)

중복된 값:
 Empty DataFrame
Columns: [review, sentiment]
Index: []


In [10]:
# 리뷰를 소문자로 변환
data['review'] = data['review'].str.lower()

data.head()

review sentiment
0  one of the other reviewers has mentioned that ...  positive
1  a wonderful little production. <br /><br />the...  positive
2  i thought this was a wonderful way to spend ti...  positive
3  basically there's a family where a little boy ...  negative
4  petter mattei's "love in the time of money" is...  positive

In [11]:
import re

change = {
    r"\'m": " am",
    r"\'s": " is",
    r"n\'t": " not",
    r"\'re": " are",
    r"\'ve": " have",
    r"\'ll": " will",
    r"\'d": " had"

}

#text 컬럼에서 대체 작업 수행
def replace_contractions(text):
    for pattern, replacement in change.items():
        text = re.sub(pattern, replacement, text)
    return text

    # 리뷰에 적용
data['review'] = data['review'].apply(replace_contractions)

data.head()

review sentiment
0  one of the other reviewers has mentioned that ...  positive
1  a wonderful little production. <br /><br />the...  positive
2  i thought this was a wonderful way to spend ti...  positive
3  basically there is a family where a little boy...  negative
4  petter mattei is "love in the time of money" i...  positive

In [12]:
from bs4 import BeautifulSoup

# BeautifulSoup을 사용하여 HTML 태그 제거
#HTML 태그란 웹 페이지의 구조를 정의하고 내용을 표시하는 데 사용되는 마크업 언어의 요소
#보통 <>로 이루어져 있음

def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
## text에서 대괄호 [] 사이에 있는 모든 텍스트를 제거
def remove_between_square_and_parentheses(text):
    # 대괄호 [] 안의 텍스트 제거
    text = re.sub(r'\[[^\]]*\]', '', text)
    # 소괄호 () 안의 텍스트 제거
    text = re.sub(r'\([^)]*\)', '', text)
    return text


#Removing the noisy text
## 텍스트에서 HTML 태그와 대괄호 안의 노이즈 텍스트를 제거
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_and_parentheses(text)
    return text

#Apply function on review column
## 각 리뷰에서 HTML 태그와 대괄호 안의 노이즈 텍스트를 제거
data['review']=data['review'].apply(denoise_text)

data.head()
#결론 html 태그 제거, 소괄호, 대괄호 안에 있는 텍스트 제거
#노이즈 텍스트 방법이 URL 제거, 이메일 주소 제거, 이모지 제거, 특수 문자 제거, 여러 공백 처리가 있는 것임

<ipython-input-12-bbb662afaa44>:8: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


review sentiment
0  one of the other reviewers has mentioned that ...  positive
1  a wonderful little production. the filming tec...  positive
2  i thought this was a wonderful way to spend ti...  positive
3  basically there is a family where a little boy...  negative
4  petter mattei is "love in the time of money" i...  positive

In [13]:
# 영문 알파벳과 공백을 제외한 모든 문자를 제거
data['review'] = data['review'].str.replace(r'[^a-zA-Z\s]', '', regex=True)

data.head()

review sentiment
0  one of the other reviewers has mentioned that ...  positive
1  a wonderful little production the filming tech...  positive
2  i thought this was a wonderful way to spend ti...  positive
3  basically there is a family where a little boy...  negative
4  petter mattei is love in the time of money is ...  positive

In [14]:
chat_words = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk",
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart",
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet",
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously",
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

In [15]:
# 줄임말이 포함된 리뷰를 찾는 함수
def contains_abbreviation(text):
    words = re.findall(r'\b\w+\b|[^\w\s]', text)
    for w in words:
        if w.lower() in chat_words:
            return True
    return False

# 줄임말이 포함된 리뷰 필터링
abbreviations_in_reviews = data[data['review'].apply(contains_abbreviation)]

# 결과 출력
print(abbreviations_in_reviews)

                                                  review sentiment
4      petter mattei is love in the time of money is ...  positive
12     so im not a big fan of boll is work but then a...  negative
18     i remember this filmit was the first film i ha...  positive
34     i watched this film not really expecting much ...  negative
39     after sitting through this pile of dung my hus...  negative
...                                                  ...       ...
49951  scientists at a remote lab experiment on  and ...  negative
49952  if you played spiderman on the ps version then...  positive
49955  i have not seen this movie but i must having r...  positive
49959  my thoughts on the movie it was not good not g...  negative
49968  i must say as a girl with a cowboy of my owni ...  positive

[5932 rows x 2 columns]


In [16]:
data['review'].iloc[4]

'petter mattei is love in the time of money is a visually stunning film to watch mr mattei offers us a vivid portrait about human relations this is a movie that seems to be telling us what money power and success do to people in the different situations we encounter this being a variation on the arthur schnitzler is play about the same theme the director transfers the action to the present time new york where all these different characters meet and connect each one is connected in one way or another to the next person but no one seems to know the previous point of contact stylishly the film has a sophisticated luxurious look we are taken to see how these people live and the world they live in their own habitatthe only thing one gets out of all these souls in the picture is the different stages of loneliness each one inhabits a big city is not exactly the best place in which human relations find sincere fulfillment as one discerns is the case with most of the people we encounterthe acti

In [17]:
import re

def chat_conversion(text):
    new_text = []
    # 단어와 구두점을 분리하는 정규식
    words = re.findall(r'\b\w+\b|[^\w\s]', text)

    for w in words:
        lower_w = w.lower()  # 소문자로 변환하여 딕셔너리에서 찾음
        if lower_w in chat_words:
            new_text.append(chat_words[lower_w])
        else:
            new_text.append(w)

    return " ".join(new_text)

data['review'] = data['review'].apply(chat_conversion)

data.head()

review sentiment
0  one of the other reviewers has mentioned that ...  positive
1  a wonderful little production the filming tech...  positive
2  i thought this was a wonderful way to spend ti...  positive
3  basically there is a family where a little boy...  negative
4  petter mattei is love in the time of money is ...  positive

In [18]:
data['review'].iloc[4]

'petter mattei is love in the time of money is a visually stunning film to watch mister mattei offers us a vivid portrait about human relations this is a movie that seems to be telling us what money power and success do to people in the different situations we encounter this being a variation on the arthur schnitzler is play about the same theme the director transfers the action to the present time new york where all these different characters meet and connect each one is connected in one way or another to the next person but no one seems to know the previous point of contact stylishly the film has a sophisticated luxurious look we are taken to see how these people live and the world they live in their own habitatthe only thing one gets out of all these souls in the picture is the different stages of loneliness each one inhabits a big city is not exactly the best place in which human relations find sincere fulfillment as one discerns is the case with most of the people we encounterthe 

In [19]:
import nltk
from nltk.tokenize import word_tokenize

# punkt 토크나이저 모델 다운로드
nltk.download('punkt')

def tokenize_text(text):
    # 텍스트를 단어 단위로 토큰화
    tokens = word_tokenize(text)
    return tokens

# 정제된 텍스트에 대해 토큰화 적용
data['review'] = data['review'].apply(tokenize_text)

# 변환된 데이터프레임의 상위 5개 행을 표시
data.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


review sentiment
0  [one, of, the, other, reviewers, has, mentione...  positive
1  [a, wonderful, little, production, the, filmin...  positive
2  [i, thought, this, was, a, wonderful, way, to,...  positive
3  [basically, there, is, a, family, where, a, li...  negative
4  [petter, mattei, is, love, in, the, time, of, ...  positive

In [20]:
import nltk

from nltk.corpus import stopwords

# stopwords 데이터 다운로드
nltk.download('stopwords')

# 불용어 집합 정의
stop_words = set(stopwords.words('english'))

def remove_stopwords(tokens):
    # 불용어를 토큰 리스트에서 제거
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    return filtered_tokens

# 토큰화된 텍스트에서 불용어 제거 적용
data['review'] = data['review'].apply(remove_stopwords)

# 변환된 데이터프레임의 상위 5개 행을 표시
data.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


review sentiment
0  [one, reviewers, mentioned, watching, oz, epis...  positive
1  [wonderful, little, production, filming, techn...  positive
2  [thought, wonderful, way, spend, time, hot, su...  positive
3  [basically, family, little, boy, thinks, zombi...  negative
4  [petter, mattei, love, time, money, visually, ...  positive

In [21]:
from collections import Counter
from nltk import FreqDist
import numpy as np

In [22]:
from gensim.models import Word2Vec

# Word2Vec 모델 학습
model = Word2Vec(sentences=data['review'], vector_size=100, window=5, min_count=1, workers=4)

# 학습된 단어 임베딩 벡터 확인
word_embedding = model.wv['first']  # 'first' 단어의 벡터
print("Word2Vec embedding for 'example':", word_embedding)

Word2Vec embedding for 'example': [ 1.0769416   0.39601392 -0.9367441  -0.5072309   1.1021242   0.70993215
  1.3991979  -0.1417716  -1.119976    1.4521227   0.19264546 -0.92066085
 -0.45755336  1.1775811   2.2448623  -0.43305725  0.34911796 -0.266924
 -1.1956515  -3.032602   -2.1843636   1.2852836   1.5566053  -0.14033121
 -0.8368119   1.9542505  -0.6208438  -0.79339707  0.4031927   2.6188028
  1.913199   -1.9955395  -0.52688116  0.77533835 -3.082623    1.5752889
 -0.13021153 -0.66146904 -2.2580414  -0.870628   -0.8870404   1.3721869
  3.1087575   0.8491716   0.70955473 -1.4251709   0.95548606 -1.3636209
  2.014844   -0.5800106  -1.8994062  -0.7555095  -1.8373449   1.310222
  2.1234787   2.487003    0.6980526   1.1174555   0.3037247  -0.0957348
 -0.75089085 -0.7647513  -1.0769104  -1.9744748  -2.0074093   0.7285197
 -1.5067458  -1.7093074  -1.1010562  -0.64991707 -0.7612929   0.6120684
  1.8967745  -0.0708724   0.11596939  0.8785741  -0.92898494  3.293749
  0.6167374  -3.4537873  -1.54

In [23]:
model.wv.most_similar('first')

[('second', 0.7627344727516174),
 ('third', 0.6068847179412842),
 ('last', 0.59661865234375),
 ('assedin', 0.5614927411079407),
 ('nd', 0.5444480776786804),
 ('placeadding', 0.543634295463562),
 ('facehuggerqueen', 0.5340365767478943),
 ('truthsthe', 0.5327586531639099),
 ('fourth', 0.530705451965332),
 ('initial', 0.5253894925117493)]